<a target="_blank" href="https://colab.research.google.com/github/Nick24-hub/Advanced-Topics-in-Neural-Networks-Template-2023/blob/main/Lab05/Solution/lab5_assignment.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import torch
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import wandb

In [2]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
        # For multi-gpu workstations, PyTorch will use the first available GPU (cuda:0), unless specified otherwise
        # (cuda:1).
    if torch.backends.mps.is_available():
        return torch.device('mos')
    return torch.device('cpu')

In [3]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size1)
        self.bn1 = torch.nn.BatchNorm1d(hidden_size1)
        self.fc2 = torch.nn.Linear(hidden_size1, hidden_size2)
        self.bn2 = torch.nn.BatchNorm1d(hidden_size2)
        self.fc3 = torch.nn.Linear(hidden_size2, output_size)
        self.relu = torch.nn.ReLU(inplace=True)
        self.dropout = torch.nn.Dropout(0.2)
        
    def forward(self, x):
        #return self.fc3(self.dropout(self.relu(self.bn2(self.fc2(self.dropout(self.relu(self.bn1(self.fc1(x)))))))))
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [4]:
def accuracy(output, labels):
    fp_plus_fn = torch.logical_not(output == labels).sum().item()
    all_elements = len(output)
    return (all_elements - fp_plus_fn) / all_elements

In [39]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()

    all_outputs = []
    all_labels = []
    epoch_loss = 0
    
    for batch, batch_data in enumerate(train_loader, 0):
        data, labels = batch_data;
        data = data.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        output = model(data)
        loss = criterion(output, labels)
        epoch_loss += loss.item()
        writer.add_scalar("AdaGrad/Train/Batch loss", loss.item(), batch)
    
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), 5)

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        output = output.softmax(dim=1).detach().cpu().squeeze()
        labels = labels.cpu().squeeze()
        all_outputs.append(output)
        all_labels.append(labels)

    all_outputs = torch.cat(all_outputs).argmax(dim=1)
    all_labels = torch.cat(all_labels)

    return round(accuracy(all_outputs, all_labels), 4), epoch_loss

In [40]:
def val(model, val_loader, device):
    model.eval()

    all_outputs = []
    all_labels = []
    epoch_loss = 0
    
    for data, labels in val_loader:
        data = data.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        with torch.no_grad():
            output = model(data)
            
        loss = criterion(output, labels)
        epoch_loss += loss.item()
        
        output = output.softmax(dim=1).cpu().squeeze()
        labels = labels.cpu().squeeze()
        all_outputs.append(output)
        all_labels.append(labels)

    all_outputs = torch.cat(all_outputs).argmax(dim=1)
    all_labels = torch.cat(all_labels)
    
    return round(accuracy(all_outputs, all_labels), 4), epoch_loss

In [41]:
def do_epoch(model, train_loader, val_loader, criterion, optimizer, device):
    acc, loss = train(model, train_loader, criterion, optimizer, device)
    acc_val, loss_val = val(model, val_loader, device)
    #torch.cuda.empty_cache()
    return acc, acc_val, loss, loss_val

In [42]:
def get_model_norm(model):
    norm = 0.0
    for param in model.parameters():
        norm += torch.norm(param)
    return norm

In [43]:
device=get_default_device()
print(device)

transforms_test = [
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((28, 28), antialias=True),
    v2.Grayscale(),
    torch.flatten,
]

transforms_train = [
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((28, 28), antialias=True),
    v2.Grayscale(),
    v2.RandAugment(),
    torch.flatten,
]

data_path = './data'
train_dataset = CIFAR10(root=data_path, train=True, transform=v2.Compose(transforms_train), download=True)
val_dataset = CIFAR10(root=data_path, train=False, transform=v2.Compose(transforms_test), download=True)

model = MLP(784, 200, 50, 10)
model = model.to(device)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
#optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

cuda
Files already downloaded and verified
Files already downloaded and verified


In [44]:
epochs = 50
batch_size = 256
val_batch_size = 500
num_workers = 2
persistent_workers = (num_workers != 0)
pin_memory = device.type == 'cuda'
train_loader = DataLoader(train_dataset, shuffle=True, pin_memory=pin_memory, num_workers=num_workers,
                          batch_size=batch_size, drop_last=True, persistent_workers=persistent_workers)
val_loader = DataLoader(val_dataset, shuffle=False, pin_memory=True, num_workers=0, batch_size=val_batch_size,
                        drop_last=False)

In [15]:
# Load the TensorBoard notebook extension

%load_ext tensorboard
%tensorboard --logdir='./runs' --port 6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [45]:
writer = SummaryWriter()
tbar = tqdm(tuple(range(epochs)))
for epoch in tbar:
    acc, acc_val, loss, loss_val = do_epoch(model, train_loader, val_loader, criterion, optimizer, device)
    current_learning_rate = optimizer.param_groups[0]['lr']
    tbar.set_postfix_str(f"Acc: {acc}, Acc_val: {acc_val}")
    writer.add_scalar("AdaGrad/Train/Accuracy", acc, epoch)
    writer.add_scalar("AdaGrad/Train/Loss", loss, epoch)
    writer.add_scalar("AdaGrad/Val/Accuracy", acc_val, epoch)
    writer.add_scalar("AdaGrad/Val/Loss", loss_val, epoch)
    writer.add_scalar("AdaGrad/Model/Norm", get_model_norm(model), epoch)
    writer.add_scalar("AdaGrad/Optimizer/Learning rate", current_learning_rate, epoch)
    writer.add_scalar("AdaGrad/Optimizer/Batch size", batch_size, epoch)

100%|█████████████████████████████████████████████████████| 50/50 [13:10<00:00, 15.80s/it, Acc: 0.394, Acc_val: 0.4502]
